In [ ]:
import cv2
from ultralytics import YOLO

# Load the pre-trained YOLO model
model = YOLO(r"C:\Users\alie0\Downloads\best (2).pt")  # Update path as needed

# Print model class names to verify
print("Model classes:", model.names)

# Define the specific classes to detect by their names
target_class_names = ["deodorant", "vitamin"]
# Map class names to their indices
target_classes = [index for index, name in model.names.items() if name in target_class_names]
print("Target class indices:", target_classes)

# Initialize the camera
cap = cv2.VideoCapture(0)

# Check if the camera is opened successfully
if not cap.isOpened():
    print("Error: Could not open the camera.")
    exit()

# List to store detected objects (names and confidence)
detected_objects = []

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    # Resize the frame to 640x640 (YOLO default size)
    frame_resized = cv2.resize(frame, (640, 640))

    # Perform YOLO inference
    results = model(frame_resized, conf=0.8, classes=target_classes)  # Lower confidence threshold

    # Get the first result
    result = results[0]

    # Print the full result for debugging
    print("Result:", result)

    # Annotate the frame with the results
    annotated_frame = result.plot()  # Adds bounding boxes, labels, etc.

    # Check if there are detections
    if len(result.boxes.cls) > 0:
        print("Detected classes and their confidences:")
        for box, score, cls in zip(result.boxes.xywh, result.boxes.conf, result.boxes.cls):
            class_name = model.names[int(cls)]  # Get class name
            confidence = float(score)  # Get confidence score
            print(f"Class: {class_name}, Confidence: {confidence:.2f}")

            # Only add target classes to the list
            if class_name in target_class_names:
                detected_objects.append((class_name, confidence))
                print(f"Detected target class: {class_name} with confidence: {confidence:.2f}")
            else:
                print(f"Detected non-target class: {class_name} with confidence: {confidence:.2f}")
    else:
        print("No target objects detected.")

    # Display the annotated frame
    cv2.imshow("YOLO Object Detection", annotated_frame)

    # Wait for the 'q' key to be pressed, and break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("\n--- Detected Objects ---")
        for obj in detected_objects:
            print(f"Class: {obj[0]}, Confidence: {obj[1]:.2f}")
        break

# Release the capture object and close any OpenCV windows
cap.release()
cv2.destroyAllWindows()
